In [119]:
import os
os.chdir('../eval/')

import hashlib
import time
import random
import json
import openai
import transformers
import fnmatch
import datasets
import pathlib
import torch

from functools import cache
from collections import Counter
from datasets import load_dataset
from warnings import warn
from abc import abstractmethod, ABC
from eval import tasks
from tasks import utils
from eval.utils import TokenizedDataset, complete_code
from eval.tasks.utils import evaluate, convert_to_nltk_rep
# from eval.generation import parallel_generations
from diskcache import Cache
from concurrent.futures import ThreadPoolExecutor


from torch.utils.data.dataloader import DataLoader
from transformers import StoppingCriteria, StoppingCriteriaList
from accelerate.utils import set_seed

from accelerate import Accelerator, DeepSpeedPlugin
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser

from eval import tasks
from eval.generation import parallel_generations
from eval.args import RunnerArguments, HFArguments, OAIArguments, GenerationArguments
# from eval.evaluator import HFEvaluator, OAIEvaluator
from eval.tasks import ALL_TASKS, TASK_REGISTRY

%load_ext autoreload
%reload_ext autoreload

transformers.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Opening JSON file 
# LLM generated 
f = open('../output/bigcode/starcoderplus_folio-neurosymbolic-1shot_generations_raw.json')
data = json.load(f)
f.close()

In [161]:
len(data)

182

In [177]:
# Data sample
index = 5
sample = data[index][0]
results = []

for sample in data:
    sample_split = sample[0].split('\n')
    sample_split
    length_range = range(0, len(sample_split))
    evaluate_content = []

    eval_end = False
    eval_start = False

    # Extract <EVALUATE> tag
    for i in reversed(length_range):
        if (eval_end == True and eval_start == True):
            break
        elif (sample_split[i] == '</EVALUATE>'):
            eval_end = True
        elif (sample_split[i] == '<EVALUATE>'):
            eval_start = True
        evaluate_content.append(sample_split[i])

    # Extract FOL only
    fol = []
    for line in list(reversed(evaluate_content)):
        if "FOL" in line:
            try:
                fol.append(line.split('\t')[1])
            except:
                # print(line)
                continue
            # print(line.split('\t')[1])
    fol

    # FOL Prover
    premises = fol[:-1]
    conclusion = fol[-1]

    # print(premises, conclusion)
    try:
        # print(utils.evaluate(premises, conclusion))
        results.append(utils.evaluate(premises, conclusion))
    except:
        results.append('Error')
        # print('Error')
print(results)

['Error', 'Error', 'Error', 'True', 'True', 'Uncertain', 'Error', 'Uncertain', 'Error', 'Uncertain', 'Uncertain', 'Error', 'Uncertain', 'True', 'Error', 'Error', 'True', 'Error', 'Uncertain', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'False', 'Error', 'True', 'True', 'Error', 'Error', 'Error', 'Error', 'Uncertain', 'Uncertain', 'True', 'Error', 'Error', 'Uncertain', 'Uncertain', 'Error', 'True', 'Uncertain', 'Error', 'Uncertain', 'Error', 'Uncertain', 'Uncertain', 'Error', 'Uncertain', 'Error', 'Error', 'Error', 'Error', 'Uncertain', 'Uncertain', 'True', 'Uncertain', 'True', 'Uncertain', 'Uncertain', 'Error', 'Uncertain', 'Error', 'False', 'True', 'Error', 'Error', 'False', 'Uncertain', 'Error', 'False', 'Uncertain', 'Error', 'True', 'Error', 'Error', 'Error', 'Uncertain', 'Error', 'Uncertain', 'False', 'Uncertain', 'Error', 'Uncertain', 'Error', 'Error', 'Error', 'True', 'Error', 'Error', 'Uncertain', 'Error', 'Uncertain', 'Uncertain', 'Error', '

In [178]:
len(data) == len(results)

True